In [53]:
!pip install tensorflow
# https://github.com/zephyrproject-rtos/zephyr/blob/main/samples/modules/tflite-micro/hello_world/train/train_hello_world_model.ipynb
# https://colab.research.google.com/github/instafluff/TensorFlowLiteMicro_MNIST/blob/master/mnist_model_tensorflow_lite_micro.ipynb#scrollTo=F9sgeZzVtzzd
# https://www.tensorflow.org/datasets/keras_example

In [61]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input


MODELS_DIR = 'models/'
if not os.path.exists(MODELS_DIR):
    os.mkdir(MODELS_DIR)
MODEL_TF = MODELS_DIR + 'model.keras'
MODEL_NO_QUANT_TFLITE = MODELS_DIR + 'model_no_quant.tflite'
MODEL_TFLITE = MODELS_DIR + 'model.tflite'
MODEL_TFLITE_MICRO = MODELS_DIR + 'model.cc'

In [55]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
x_train = x_train[..., None]  # (28,28,1)
x_test  = x_test[..., None]

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [56]:
model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(8, 4, activation='relu', input_shape=(28,28,1)),
  tf.keras.layers.MaxPool2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss="categorical_crossentropy",
    metrics=['accuracy']
)
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 25, 25, 8)      │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 12, 12, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_17 (Flatten)            │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 10)             │        11,530 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,666 (45.57 KB)

 Trainable params: 11,666 (45.57 KB)

 Non-trainable params: 0 (0.00 B)

In [57]:
model.fit(x_train, y_train, epochs=5, batch_size=128)
model.save(MODEL_TF)
scores = model.evaluate(x_test, y_test, verbose=2)
print("Test loss:", scores[0])
print("Test accuracy:", scores[1])

Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 23s 44ms/step - accuracy: 0.7814 - loss: 0.8402
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 43s 49ms/step - accuracy: 0.9479 - loss: 0.1850
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 13s 27ms/step - accuracy: 0.9649 - loss: 0.1279
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 15s 32ms/step - accuracy: 0.9722 - loss: 0.0982
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 13s 27ms/step - accuracy: 0.9761 - loss: 0.0823
313/313 - 1s - 4ms/step - accuracy: 0.9781 - loss: 0.0720
Test loss: 0.07199300080537796
Test accuracy: 0.9781000018119812


In [107]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

model_no_quant_tflite = converter.convert()
open(MODEL_NO_QUANT_TFLITE, "wb").write(model_no_quant_tflite)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
def representative_data_gen():
    for image in tf.data.Dataset.from_tensor_slices(x_train) \
                                  .batch(1).take(100):
        yield [image]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.target_spec.supported_types = [tf.int8]
converter.inference_input_type  = tf.uint8   # albo tf.int8
converter.inference_output_type = tf.uint8   # albo tf.int8
tflite_quant_model = converter.convert()
open(MODEL_TFLITE,'wb').write(tflite_quant_model)


Saved artifact at '/tmp/tmp7p1mf2kn'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='keras_tensor_80')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  133277878057680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133277878055376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133277878050000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133277878054800: TensorSpec(shape=(), dtype=tf.resource, name=None)
Saved artifact at '/tmp/tmprst41yin'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='keras_tensor_80')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  133277878057680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133277878055376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133277

/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


15208

In [108]:
interpreter = tf.lite.Interpreter(model_path=MODEL_TFLITE)

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Adjust the model interpreter to take 10,000 inputs at once instead of just 1
interpreter.resize_tensor_input(input_details[0]["index"], (10000, 28, 28, 1))
interpreter.resize_tensor_input(output_details[0]["index"], (10000, 10))
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]

in_scale, in_zero_point = input_details['quantization']
x_test_quant = (x_test.astype(np.float32) / in_scale + in_zero_point).round().astype(input_details['dtype'])

interpreter.set_tensor(input_details['index'], x_test_quant)

interpreter.invoke()

output_quant = interpreter.get_tensor(output_details['index'])
out_scale, out_zero_point = output_details['quantization']
output_dequant = (output_quant.astype(np.float32) - out_zero_point) * out_scale

# 8. Compute accuracy
preds = np.argmax(output_dequant, axis=1)
y_test_labels = np.argmax(y_test, axis=1)
accuracy = np.mean(preds == y_test_labels)
print("TFLite int8 Accuracy:", accuracy)

TFLite int8 Accuracy: 0.978


In [109]:
size_tf = os.path.getsize(MODEL_TF)
size_no_quant_tflite = os.path.getsize(MODEL_NO_QUANT_TFLITE)
size_tflite = os.path.getsize(MODEL_TFLITE)


print("Model.tf: %d bytes" % size_tf)
print("Model_no_quant.tflite: %d bytes" % size_no_quant_tflite)
print("Model.tflite: %d bytes" % size_tflite)

Model.tf: 165498 bytes
Model_no_quant.tflite: 49072 bytes
Model.tflite: 15208 bytes


In [113]:
!xxd -i {MODEL_TFLITE} > {MODEL_TFLITE_MICRO}

REPLACE_TEXT = MODEL_TFLITE.replace('/', '_').replace('.', '_')
!sed -i 's/'{REPLACE_TEXT}'/g_model/g' {MODEL_TFLITE_MICRO}

!cat {MODEL_TFLITE_MICRO}

unsigned char g_model[] = {
  0x20, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x00, 0x00, 0x00, 0x00,
  0x14, 0x00, 0x20, 0x00, 0x1c, 0x00, 0x18, 0x00, 0x14, 0x00, 0x10, 0x00,
  0x0c, 0x00, 0x00, 0x00, 0x08, 0x00, 0x04, 0x00, 0x14, 0x00, 0x00, 0x00,
  0x1c, 0x00, 0x00, 0x00, 0x8c, 0x00, 0x00, 0x00, 0x0c, 0x01, 0x00, 0x00,
  0x24, 0x30, 0x00, 0x00, 0x34, 0x30, 0x00, 0x00, 0xa4, 0x3a, 0x00, 0x00,
  0x03, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00,
  0xfa, 0xce, 0xff, 0xff, 0x0c, 0x00, 0x00, 0x00, 0x1c, 0x00, 0x00, 0x00,
  0x3c, 0x00, 0x00, 0x00, 0x0f, 0x00, 0x00, 0x00, 0x73, 0x65, 0x72, 0x76,
  0x69, 0x6e, 0x67, 0x5f, 0x64, 0x65, 0x66, 0x61, 0x75, 0x6c, 0x74, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x68, 0xff, 0xff, 0xff,
  0x0c, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x08, 0x00, 0x00, 0x00,
  0x6f, 0x75, 0x74, 0x70, 0x75, 0x74, 0x5f, 0x30, 0x00, 0x00, 0x00, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x9e, 0xcf, 0xff, 0xff,
  0x04, 0x